In [1]:
import pandas as pd
import numpy as np

In [2]:
pe_df = pd.read_csv("historical_pe_ratio.csv")
pe_df.set_index("Unnamed: 0", inplace=True)
eps_df = pd.read_csv("historical_eps.csv")
eps_df.set_index("Unnamed: 0", inplace=True)
price = pd.read_csv("data/sp500_close_data.csv")

In [3]:
# Will be calculated for all dates after '2016-12-31'
# Will remain same for a particular quarter

# Step 1: Check for a Stable/Unstable Earning Pattern - Reject those with unstable earnings pattern
# Step 2: Calculate the Average P/E (Intrinsic P/E)
# Step 3: Estimate the EPS Growth Rate
# Step 4: Estimate the P/E Growth Rate
# Step 5: Average PE * Last EPS = Intrinsic value lower bound 
# Step 6: Expected PE * Expected EPS = Intrinsic value upper bound

In [4]:
def preceding_date(given):
    given_date = pd.Timestamp(given)
    date_list = list(pe_df.index)
    date_list = pd.to_datetime(date_list)
    time_diffs = [(given_date - date).days for date in date_list]
    min_diff_index = min(i for i, diff in enumerate(time_diffs) if diff > 0)
    preceding_date = date_list[min_diff_index]
    return str(preceding_date.date())

In [5]:
def growth_rate(eps):
    eps_gr = (eps[-1]/eps[0])**(1/len(eps)) 
    return eps_gr

In [6]:
def stability(eps):
    mid_point = len(eps) // 2
    eps_mean_first_half = np.mean(eps[:mid_point])
    eps_mean_second_half = np.mean(eps[mid_point:])

    if (eps_mean_second_half > eps_mean_first_half) and growth_rate(eps) > 1:
        return 1
    else :
        return 0

In [7]:
def intrinsic_value(eps, pe):
    if stability(eps):

        ape = np.mean(pe)
        last_eps = eps[-1]
        epe = pe[-1]*(growth_rate(pe))
        eeps = last_eps*(growth_rate(eps))
        return [ape*last_eps, epe*eeps]
    
    else:
        return "NA"

In [8]:
def calculate_intrinsic_value(given_date):
    
    q_date = preceding_date(given_date)
    prev_eps = eps_df.loc[:q_date:-1]
    prev_pe = pe_df.loc[:q_date:-1]

    results = []
    for i in price.columns[1:]:
        results.append(intrinsic_value(prev_eps[i], prev_pe[i]))

    healthy = []

    for i in results:
        if i == 'NA':
            healthy.append("NA")
            continue
        elif i[0] > 0 and i[1] > 0:
            healthy.append(i)
        else:
            healthy.append("NA")
    
    return healthy

In [9]:
given_date = "2018-09-11"
IV = calculate_intrinsic_value(given_date)

/var/folders/yl/5879yj014q11kbqcnjh_rd0m0000gn/T/ipykernel_20430/3699704206.py:2: RuntimeWarning: invalid value encountered in double_scalars
  eps_gr = (eps[-1]/eps[0])**(1/len(eps))


In [10]:
for i in range(len(price.columns[1:])):
    if IV[i] != 'NA':
        print(price.columns[i])

Date
AAPL
ABT
ACN
ADBE
ADP
AXP
BAC
BKNG
BLK
CB
CI
COP
CVX
DHR
DIS
ELV
ETN
GE
GOOG
GOOGL
GS
IBM
INTU
JNJ
JPM
LLY
LMT
LRCX
MA
MDT
META
MRK
MS
MSFT
MU
NEE
NKE
NOW
PEP
PG
PGR
QCOM
RTX
SBUX
SNPS
SPGI
SYK
T
TJX
TSLA
TXN
UNH
UNP
UPS
VRTX
